In [465]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import sys
from scipy import stats
from sklearn.neighbors import KDTree

In [466]:
im = cv2.imread('./images/0.jpg', cv2.IMREAD_COLOR)
im_gray = cv2.imread('./images/0.jpg', cv2.IMREAD_GRAYSCALE)

In [467]:
# im_color = cv2.resize(im, (0,0), fx=1/2, fy=1/2)
im_color = im

In [468]:
def get_gauss_pyramid(im, l=4, sub=2, smoothing=1.0):
    res = [im]
    
    for i in range(l):
        tmp = cv2.GaussianBlur(im, (0,0), smoothing)
        tmp = cv2.resize(im, (0,0), fx=1/sub, fy=1/sub)
        res.append(tmp)
        im = tmp
    
    return res

In [469]:
def get_harris_matrix(im, si=1.5, sd=1, k=0.04):
    h, w = im.shape[0], im.shape[1]
    prod = []
    
    Iy, Ix = np.gradient(im)
    
    Ix2 = np.square(Ix)
    Iy2 = np.square(Iy)
    Ixy = Ix * Iy
    
    Sx2 = cv2.GaussianBlur(Ix2, (3, 3), si)
    Sy2 = cv2.GaussianBlur(Iy2, (3, 3), si)
    Sxy = cv2.GaussianBlur(Ixy, (3, 3), si)
    
    detM = (Sx2 * Sy2) - (np.square(Sxy))
    trM = Sx2 + Sy2

    R = detM - k * (np.square(trM))
        
    return R, Sx2, Sy2, Sxy

In [470]:
def get_max_R(R, threshold=0.1, step=12):
    localMax = np.ones(R.shape, dtype=np.uint8)
    localMax[R <= np.max(R) * threshold] = 0
    maxima = []
    
    for i in range(0, R.shape[0]-step, step):
        for j in range(0, R.shape[1]-step, step):
            l_max = R[i, j]
            max_coord = (i, j)
            for k in range(step):
                for l in range(step):
                    c = localMax[i+k, j+l]
                    if (c == 1) and (R[i+k, j+l] > l_max):
                        l_max = R[i+k, j+l]
                        max_coord = (i+k, j+l)
#                         print(max_coord)

            if max_coord == (i, j):
                if localMax[i, j] == 1:
                    maxima.append((max_coord[1], max_coord[0]))
            else:
                maxima.append((max_coord[1], max_coord[0]))
    
    return maxima

In [471]:
def get_feature_points(im):
    R, Sx2, Sy2, Sxy = get_harris_matrix(im)
    maxima = get_max_R(R)
    
#     for p in maxima:
#         output_im = cv2.circle(output_im, p, 1, (0, 0, 255), 1)
    
    return maxima

In [472]:
def get_orientations(M, so=4.5):
    Iy, Ix = np.gradient(M)
    Sy, Sx = cv2.GaussianBlur(Iy, (3, 3), so), cv2.GaussianBlur(Ix, (3, 3), so)
#     print(Sy.shape)
    res = []
    
    for i in range(Sy.shape[0]):
        tmp = []
        for j in range(Sy.shape[1]):
            v = np.array([Sx[i, j], Sy[i, j]])
            v_norm = np.linalg.norm(v)
            if v_norm == 0:
                tmp.append(np.array([0.0, 0.0]))
            else:
                tmp.append(v / v_norm)
        res.append(tmp)
    
    return np.array(res)

In [473]:
def get_angles(Z):
    angles = np.zeros((Z.shape[0], Z.shape[1]))
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            x = Z[i, j]
            angles[i, j] = (int(np.degrees(np.arctan2(x[1], x[0])))+360)%360
    
    return angles

In [474]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
#     print(rot_mat)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [475]:
def get_patch(image, p, angle, s=8):
    p = np.array([[p[1]],[p[0]]])
    height, width = image.shape[:2]
    image_center = (width/2, height/2)
    
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    abs_cos = abs(rot_mat[0,0])
    abs_sin = abs(rot_mat[0,1])

    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    rot_mat[0, 2] += bound_w/2 - image_center[0]
    rot_mat[1, 2] += bound_h/2 - image_center[1]
    
    rot_img = cv2.warpAffine(image, rot_mat, (bound_w, bound_h), flags=cv2.INTER_LINEAR)
    A = [
        [rot_mat[0][0], rot_mat[0][1]],
        [rot_mat[1][0], rot_mat[1][1]],
        ]
    
    B = [
        [rot_mat[0][2]],
        [rot_mat[1][2]],
    ]
    new_p = np.matmul(A, p) + B
    new_i = int(new_p[0])
    new_j = int(new_p[1])
    
    res = []
    r = int(s/2)
    for j in range(-r, r, 1):
        tmp = []
        for i in range(-r, r, 1):
            try:
                tmp.append(rot_img[new_j+j, new_i+i])
            except:
                tmp.append(0.0)
        res.append(tmp)

    res = np.array(res)
    if np.isnan(res).any():
        print(res)
    
    return res

In [500]:
def get_description_vectors(img, feature_points, angles, size=8):
    left_dv = []
    right_dv = []
    
    width_mid = int(img.shape[1] / 2)
    for i, p in enumerate(feature_points):        
        p = (p[1], p[0])
        angle = angles[p[0], p[1]]
        if angle >= 0 and angle < 90:
            angle = 360-abs(270+angle)
        else:
            angle = 360-abs(90-angle)
        vec = get_patch(img, p, angle, size).flatten()
        if(np.sum(vec) == 0):
            print("sum is 0")
        norm_vec = np.array(stats.zscore(vec))
        if p[1] < width_mid:
            left_dv.append(norm_vec)
        else:
            right_dv.append(norm_vec)
    dv = [np.array(left_dv), np.array(right_dv)]
    
    if np.isnan(np.array(left_dv)).any():
        print(np.array(left_dv))
    
    return dv

In [477]:
def get_all_img_DV(images):
    res = []
    
    for i, image in enumerate(images):
        print("processing image {}".format(i))
        im_pyr = get_gauss_pyramid(image)
        img = im_pyr[0]
        
        fp = get_feature_points(img)
        
        z = get_orientations(img)
        angles = get_angles(z)
        
        d_vecs = get_description_vectors(img, fp, angles)
        
        res.append(d_vecs)
        
    return res

In [ ]:
def find_neighbor_images(img):
    

In [478]:
images = []

for i in range(5):
    images.append(cv2.imread('./images/{}.jpg'.format(i), cv2.IMREAD_GRAYSCALE))

images = np.array(images)

In [501]:
# DV structure:
# Array for each image. Each array has 2 arrays, left and right
# each of those arrays has N 64 length vectors representing patches

DV = get_all_img_DV(images)

processing image 0
processing image 1
processing image 2
processing image 3
processing image 4


In [493]:
type(DV[0][0])

numpy.ndarray

In [502]:
for i in DV:
    if np.isnan(i[0]).any():
        print(i[0])

In [503]:
# left and right patches of all images

DV_left = [x[0] for x in DV]
DV_right = [x[1] for x in DV]

# DV_left = DV[:, 0]
# DV_right = DV[:, 1]

# DV_left = DV_left[~np.isnan(DV_left)]
# DV_right = DV_right[~np.isnan(DV_right)]

In [518]:
len(DV_l)

5

In [483]:
DV_left = DV_left[:, ~np.isnan(DV_left).any(axis=0)]
DV_right = DV_right[:, ~np.isnan(DV_right).any(axis=0)]
# for c, i in enumerate(DV_left):
#     DV_left[c] = i[~np.isnan(i)]
    
# for c, i in enumerate(DV_right):
#     DV_right[c] = i[~np.isnan(i)]
#     DV_right = DV_right[~np.isnan(DV_right)]

c:\users\nikita\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [514]:
DV_right[4].shape

(168, 64)

In [515]:
test = np.array(DV_right[4])
tree = KDTree(test)
res = []
avg_dist = []


for i, img_dv in enumerate(DV_left):
#     print(img_dv)
#     print(np.array(img_dv).shape)
    r = tree.query(img_dv)
    res.append(r)
    avg_dist.append(np.mean(r[0]))

# res = np.array(res)
# avg_dist = np.array(avg_dist)

In [516]:
avg_dist

[6.584941925983713,
 6.436888075769845,
 6.227114803050381,
 5.763450487818598,
 6.152960791391464]

In [ ]:
avg_dist

In [ ]:
img = images[0]
fp = get_feature_points(img)
fp[0]

In [ ]:
im_color = cv2.imread('./images/1.jpg', cv2.IMREAD_COLOR)

In [ ]:
img = images[1]
fp = get_feature_points(img)
z = get_orientations(img)
angles = get_angles(z)

In [ ]:
fp[1]

In [ ]:
for i, p in enumerate(fp):
    angle = angles[p[1], p[0]]
    
    x2 = int(p[0] + 8 * np.cos(np.radians((360-angle))))
    y2 = int(p[1] + 8 * np.sin(np.radians((360-angle))))
    
    cv2.line(im_color, (p[0], p[1]), (x2, y2), (0, 0, 255), thickness=1, lineType=1)

plt.imshow(im_color)
cv2.imwrite("./images/test.png", im_color)